In [1]:
from google.colab import drive
drive.mount("/content/drive")
!pwd
!ls /content/drive/MyDrive/Alzheimer_s\ Dataset.zip

Mounted at /content/drive
/content
'/content/drive/MyDrive/Alzheimer_s Dataset.zip'


In [2]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/Alzheimer_s Dataset.zip"

extracted_dir = "/content/Alzheimer_s_Dataset"  # Define the directory where files will be extracted

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Get the path to the inner directory containing the dataset files
inner_dir = os.path.join(extracted_dir, 'Alzheimer_s Dataset')

# Define paths to train data directory
train_dir = os.path.join(inner_dir, 'train')

# Check if the train directory exists
if not os.path.exists(train_dir):
    print("Train directory not found:", train_dir)
    exit()

# Define image size and batch size
img_size = (224, 224)
batch_size = 30

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Additional rotation
    width_shift_range=0.2,  # Additional horizontal shift
    height_shift_range=0.2,  # Additional vertical shift
    fill_mode='nearest'  # Fill in missing pixels with the nearest available value
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'  # Assuming images are organized in folders by class
)

# Load MobileNet model without top (classification) layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers with regularization
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Dropout for regularization
    Dense(512, activation='relu'),  # Reduced number of neurons
    Dropout(0.5),  # Dropout for regularization
    Dense(256, activation='relu'),  # Further reduced number of neurons
    BatchNormalization(),  # Batch normalization layer
    Dense(4, activation='softmax')
])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fine-tune the last 6 layers
for layer in base_model.layers[:-6]:
    layer.trainable = False

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model with fine-tuning
fine_tune_history = model.fit(train_generator, epochs=50, callbacks=[early_stopping])

# Save the fine-tuned model
model.save("/content/drive/MyDrive/MN_fine_tuned_model.h5")


Found 5121 images belonging to 4 classes.
17225924/17225924 [==============================] - 0s 0us/step
Epoch 1/50
171/171 [==============================] - ETA: 0s - loss: 1.9165 - accuracy: 0.2726

171/171 [==============================] - 89s 455ms/step - loss: 1.9165 - accuracy: 0.2726
Epoch 2/50
171/171 [==============================] - ETA: 0s - loss: 1.8781 - accuracy: 0.2734

171/171 [==============================] - 74s 434ms/step - loss: 1.8781 - accuracy: 0.2734
Epoch 3/50
171/171 [==============================] - ETA: 0s - loss: 1.7536 - accuracy: 0.2939

171/171 [==============================] - 73s 427ms/step - loss: 1.7536 - accuracy: 0.2939
Epoch 4/50
171/171 [==============================] - ETA: 0s - loss: 1.7097 - accuracy: 0.3185

171/171 [==============================] - 73s 428ms/step - loss: 1.7097 - accuracy: 0.3185
Epoch 5/50
171/171 [==============================] - ETA: 0s - loss: 1.6650 - accuracy: 0.3300

...

171/171 [==============================] - 75s 439ms/step - loss: 1.0184 - accuracy: 0.5614
Epoch 48/50
171/171 [==============================] - ETA: 0s - loss: 1.0167 - accuracy: 0.5626

171/171 [==============================] - 74s 431ms/step - loss: 1.0167 - accuracy: 0.5626
Epoch 49/50
171/171 [==============================] - ETA: 0s - loss: 1.0262 - accuracy: 0.5552

171/171 [==============================] - 74s 433ms/step - loss: 1.0262 - accuracy: 0.5552
Epoch 50/50
171/171 [==============================] - ETA: 0s - loss: 1.0099 - accuracy: 0.5690

171/171 [==============================] - 74s 432ms/step - loss: 1.0099 - accuracy: 0.5690


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/Alzheimer_s Dataset.zip"

extracted_dir = "/content/Alzheimer_s_Dataset"  # Define the directory where files will be extracted

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Get the path to the inner directory containing the dataset files
inner_dir = os.path.join(extracted_dir, 'Alzheimer_s Dataset')

# Define paths to test data directory
test_dir = os.path.join(inner_dir, 'test')

# Check if the test directory exists
if not os.path.exists(test_dir):
    print("Test directory not found:", test_dir)
    exit()

# Load the fine-tuned model
model = load_model("/content/drive/MyDrive/MN_fine_tuned_model.h5")

# Define image size and batch size for testing
img_size = (224, 224)
batch_size = 30

# Data preprocessing for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming images are organized in folders by class
    shuffle=False  # Don't shuffle to maintain order for evaluation
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels for the test data
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
true_labels = test_generator.classes

# Generate classification report
class_names = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(true_labels, y_pred_classes, target_names=class_names))



Found 1279 images belonging to 4 classes.
43/43 [==============================] - 4s 79ms/step - loss: 0.9529 - accuracy: 0.5653
Test Loss: 0.9529249668121338
Test Accuracy: 0.5652853846549988
43/43 [==============================] - 3s 48ms/step

Classification Report:
                  precision    recall  f1-score   support

    MildDemented       0.20      0.01      0.01       179
ModerateDemented       0.00      0.00      0.00        12
     NonDemented       0.79      0.55      0.65       640
VeryMildDemented       0.45      0.83      0.58       448

        accuracy                           0.57      1279
       macro avg       0.36      0.35      0.31      1279
    weighted avg       0.58      0.57      0.53      1279



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
